J. An, Y. Chen and H. Shin, "Weather Classification using Convolutional Neural Networks," 2018 International SoC Design Conference (ISOCC), Daegu, Korea (South), 2018, pp. 245-246, doi: 10.1109/ISOCC.2018.8649921.

https://blog.csdn.net/mengxianglong123/article/details/127330721

https://www.cnblogs.com/miraclepbc/p/14348160.html

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
import os
import re
from PIL import Image
import numpy as np
import torch
from torchvision import transforms
from torch.utils.data import Dataset

class WeatherDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.image_files = os.listdir(data_dir)
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((224, 224)),  # Resize images to a consistent size
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize image data
        ])

        self.class_names = ['Sunny', 'Snowy', 'Cloudy', 'Rainy', 'Foggy']

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_name = self.image_files[idx]
        image_path = os.path.join(self.data_dir, image_name)
        image = Image.open(image_path).convert('RGB')
        image = self.transform(image)

        # Extract label from image name
        label = re.match(r'^[A-Za-z]+', image_name).group(0)

        # Convert label to numerical value
        label_mapping = {
            'Sunny': 0,
            'Snowy': 1,
            'Cloudy': 2,
            'Rainy': 3,
            'Foggy': 4
        }
        label = label_mapping[label]

        # Convert label to one-hot encoding
        # label_onehot = torch.nn.functional.one_hot(torch.tensor(label), num_classes=5)

        return image, label #label_onehot



# Set the path to your image data directory
data_dir = '/content/drive/MyDrive/Colab_Notebooks/5002/DSAA5002_50015756_Gelin_Bian_final/dsaa5002_2023fall_final_datasets/Data_Q2/train_data'

# Create the dataset instance
dataset = WeatherDataset(data_dir)


train_set = dataset
# Create data loaders
batch_size = 16
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)


In [23]:
from collections import defaultdict

class_counts = defaultdict(int)
class_names = ['Sunny', 'Snowy', 'Cloudy', 'Rainy', 'Foggy']

# 遍历数据集
for image, label_onehot in dataset:
    # 找到类别索引
    class_index = torch.argmax(label_onehot).item()
    # 统计样本数量
    class_counts[class_index] += 1



In [24]:
for key, value in class_counts.items():
    print(f"Class: {key}, Count: {value}")

Class: 2, Count: 50
Class: 4, Count: 50
Class: 3, Count: 50
Class: 1, Count: 50
Class: 0, Count: 50


In [7]:
import torch
from torch import nn
import torchvision.models as models
# Load ResNet-50 with pretrained weights
resnet = models.resnet50(pretrained=True)
class WeatherModel(nn.Module):
    def __init__(self, net):
        super(WeatherModel, self).__init__()
        # resnet50
        self.net = resnet
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(1000, 8)
        self.output = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.net(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc(x)
        x = self.output(x)
        return x
model = WeatherModel(resnet)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 147MB/s]


In [8]:

import torch.optim as optim
from torch.utils.data import DataLoader

# Set the device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set up the training loop
def train(model, train_loader, criterion, optimizer, num_epochs):
    model.to(device)
    model.train()  # Set the model to training mode

    for epoch in range(num_epochs):
        running_loss = 0.0

        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")



# Set up the model, loss function, and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 50
train(model, train_loader, criterion, optimizer, num_epochs)


Epoch [1/50], Loss: 2.0760
Epoch [2/50], Loss: 1.9699
Epoch [3/50], Loss: 1.7706
Epoch [4/50], Loss: 1.5984
Epoch [5/50], Loss: 1.4486
Epoch [6/50], Loss: 1.3725
Epoch [7/50], Loss: 1.3403
Epoch [8/50], Loss: 1.3279
Epoch [9/50], Loss: 1.3113
Epoch [10/50], Loss: 1.3171
Epoch [11/50], Loss: 1.3087
Epoch [12/50], Loss: 1.2971
Epoch [13/50], Loss: 1.2987
Epoch [14/50], Loss: 1.2960
Epoch [15/50], Loss: 1.2884
Epoch [16/50], Loss: 1.2838
Epoch [17/50], Loss: 1.2839
Epoch [18/50], Loss: 1.2800
Epoch [19/50], Loss: 1.2794
Epoch [20/50], Loss: 1.2780
Epoch [21/50], Loss: 1.2773
Epoch [22/50], Loss: 1.2787
Epoch [23/50], Loss: 1.2784
Epoch [24/50], Loss: 1.2803
Epoch [25/50], Loss: 1.2828
Epoch [26/50], Loss: 1.2798
Epoch [27/50], Loss: 1.2784
Epoch [28/50], Loss: 1.2773
Epoch [29/50], Loss: 1.2776
Epoch [30/50], Loss: 1.2825
Epoch [31/50], Loss: 1.2766
Epoch [32/50], Loss: 1.2763
Epoch [33/50], Loss: 1.2808
Epoch [34/50], Loss: 1.2757
Epoch [35/50], Loss: 1.2798
Epoch [36/50], Loss: 1.2793
E

In [9]:
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total * 100
    print(f"Accuracy: {accuracy:.2f}%")



In [10]:
evaluate(model, train_loader)

Accuracy: 100.00%
